<a href="https://colab.research.google.com/github/Vaelastraszz/NLP_Introduction/blob/master/Articles_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


# Loading des différents articles de CNN

In [0]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, encoding='utf-8')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

Fonction pour load un fichier texte

In [0]:
# split a document into news story and highlights
def split_story(doc):
    # find first highlight
	index = doc.find('@highlight')
	# split into story and highlights
	story, highlights = doc[:index], doc[index:].split('@highlight')
	# strip extra white space around each highlight
	highlights = [h.strip() for h in highlights if len(h) > 0]
	return story, highlights

Création d'une fonction pour split chaque texte en texte normal et highlights pour ensuite remplir une liste dans laquelle on mettre les hight lights et l'article sur deux colonnes articles/highlights

In [0]:
from os import listdir

def load_stories(directory):
    stories = list()
    counter = 0
    for name in listdir(directory):
        if counter < 1000:
            filename = directory + "/" + name
            doc = load_doc(filename)
            story, highlights = split_story(doc)
            stories.append({"story":story, "highlights":highlights})
            
        else:
            break
        counter = counter + 1
    return stories

Fonction pour load tous les fichiers du dossier dans lequel les fichiers textes sont stockés pui stockage des articles et des highlights dans une liste

In [0]:
# load stories
directory = '/content/gdrive/My Drive/cnn/stories2'
stories = load_stories(directory)
print('Loaded Stories %d' % len(stories))

Loaded Stories 1000


# Nettoyage des données textuelles avec NLTK

In [0]:
import nltk

tokenizer = nltk.RegexpTokenizer(r'\w+')
#tokenizer.tokenize(stories[1]["story"])

In [0]:
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize

stop_words = set(stopwords.words('english'))
stop_words.update(['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}', 'The', 'At', 'In'])

In [0]:
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Lemmatize with POS Tag
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [0]:
def tokenization_highlighs(stories, i):
    
    new_high=[]
    
    for j in range(len(stories[i]["highlights"])):
        
        #new_high.append(tokenizer.tokenize(stories[i]["highlights"][j]))
        new_high.append([lemmatizer.lemmatize(w.lower(),get_wordnet_pos(w.lower())) 
                         for w in tokenizer.tokenize(stories[i]["highlights"][j]) if not w.lower() in stop_words])
        
    return new_high

In [0]:
def tokenize_list(stories):

    new_stories = list()
    freq =  nltk.FreqDist()
    
    for i in range(len(stories)):
        
        new_high = tokenization_highlighs(stories, i)
        new_stories.append({"story": [lemmatizer.lemmatize(w.lower(),get_wordnet_pos(w.lower())) 
                                      for w in tokenizer.tokenize(stories[i]["story"]) if not w.lower() in stop_words and str.isalpha(w)== True ], 
                            "highlights":new_high})
        
        freq += nltk.FreqDist(new_stories[i]["story"])
        
    return new_stories, freq

In [0]:
test_list,freq = tokenize_list(stories)

In [0]:
stop_words.update(list(zip(*freq.most_common(50)))[0])

# TF-IDF

<h2> TF-IDF </h2>

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
import itertools

In [0]:
vectorizer =  TfidfVectorizer()

stock=[]

for i in range(len(stories)):
    stock.append(test_list[i]["story"])
    
stock = list(itertools.chain.from_iterable(stock))    
    
vectorizer.fit(stock)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [0]:
from nltk.tokenize.moses import MosesDetokenizer

detokenizer = MosesDetokenizer()

In [0]:
vector_test = vectorizer.transform([detokenizer.detokenize(test_list[1]["story"], return_str=True)])

In [0]:
from collections import Counter

len(list(Counter(detokenizer.detokenize(test_list[3]["story"], return_str=True).split())))

337

In [0]:
def create_tdf_matrix():
    
    s = (len(stories),vector_test.shape[1])
    
    X = np.ones(s)
    
    for i in range(len(stories)):
        
        v = vectorizer.transform([detokenizer.detokenize(test_list[i]["story"], return_str=True)])
        X[i] = v.toarray()[0]
    
    return X

In [0]:
X = create_tdf_matrix()

In [0]:
X.shape
#prout = X[3]
#prout[prout>0]

(1000, 23891)

In [0]:
vectorizer.get_feature_names()

In [0]:
print(vectorizer.vocabulary_)
print(vectorizer.idf_)

{'22': 364, 'month': 14954, 'ago': 1307, 'body': 3343, 'odin': 15900, 'lloyd': 13438, 'found': 8986, 'industrial': 11407, 'park': 16530, 'north': 15696, 'attleboro': 2281, 'massachusetts': 14131, 'shot': 20300, 'seven': 20041, 'time': 22377, 'near': 15410, 'pile': 17024, 'construction': 5408, 'waste': 24016, 'semipro': 19935, 'football': 8885, 'player': 17129, 'boston': 3475, 'bandit': 2612, '27': 409, 'day': 6131, 'dead': 6152, 'nfl': 15525, 'star': 21163, 'aaron': 862, 'hernandez': 10546, 'arrest': 2074, 'charge': 4495, 'murder': 15166, 'testimony': 22160, 'case': 4264, 'begin': 2874, 'january': 12024, 'wednesday': 24098, 'fall': 8367, 'river': 19030, 'jury': 12299, 'deliberate': 6344, '35': 483, 'hour': 10856, 'course': 5659, 'former': 8957, 'england': 7788, 'patriot': 16633, 'tight': 22348, 'end': 7745, 'guilty': 10001, 'degree': 6320, 'carry': 4238, 'penalty': 16745, 'without': 24375, 'parole': 16545, 'unlawful': 23235, 'possession': 17350, 'firearm': 8691, 'ammunition': 1667, 'fa

<h2> LDA + non negative matrix factorisation </h2>

In [0]:
from sklearn.decomposition import NMF

In [0]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic %d:" % (topic_idx))
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [0]:
no_topics = 20

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(X)

no_top_words = 10
display_topics(nmf, vectorizer.get_feature_names(), no_top_words)

Topic 0:
say go accord year take make statement month call include
Topic 1:
team game season player second league club goal win score
Topic 2:
obama republican house senate democrat party election vote congress political
Topic 3:
al syria syrian isi assad iraq qaeda force group opposition
Topic 4:
north korea korean south pyongyang kim nuclear tension seoul jong
Topic 5:
court case justice supreme attorney judge charge federal trial prosecutor
Topic 6:
plane airport flight passenger aircraft pilot airline air aviation jet
Topic 7:
marriage gay sex couple law moore court parent gender republican
Topic 8:
airline pilot flight seat write fly aa loose passenger lubitz
Topic 9:
flood storm rain tornado water weather area inch damage tropical
Topic 10:
pakistan attack afghanistan taliban afghan pakistani militant say troop kill
Topic 11:
china chinese beijing tibetan japan tibet island taiwan japanese protest
Topic 12:
child parent mother boy girl kid school son old love
Topic 13:
match seed

In [0]:
test_list[70]["story"]

In [0]:
#np.argmax(nmf.transform([X[1]]))
nmf.transform([X[70]])

array([[0.08235653, 0.        , 0.        , 0.15130781, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ]])

# Downoad packages NLTK

In [0]:
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('perluniprops')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.


True

In [0]:
test_list[15]["highlights"]

[['displace', 'live', 'moscow', 'lose', 'home', 'live', 'great', 'boston'],
 ['wfp', 'cut', 'food', 'ration', 'size', '30', 'lack', 'donation']]